<a href="https://colab.research.google.com/github/thotla-vandith/DL-Lesson5/blob/master/DL_ICP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
# data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_csv('drive/My Drive/Python Colab/DL_Lesson5/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [0]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

In [0]:
def createmodel():
    model = Sequential()
    lstm_out=196
    embed_dim=128
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    #print(model.summary())

In [25]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
print(integer_encoded)
print(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [26]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print('score',score)
print('accuracy',acc)
print('Model Metrics Names',model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 17s - loss: 0.8293 - accuracy: 0.6446
Epoch 2/20
 - 16s - loss: 0.6817 - accuracy: 0.7123
Epoch 3/20
 - 17s - loss: 0.6191 - accuracy: 0.7397
Epoch 4/20
 - 17s - loss: 0.5728 - accuracy: 0.7586
Epoch 5/20
 - 17s - loss: 0.5320 - accuracy: 0.7817
Epoch 6/20
 - 17s - loss: 0.4972 - accuracy: 0.7968
Epoch 7/20
 - 16s - loss: 0.4601 - accuracy: 0.8114
Epoch 8/20
 - 17s - loss: 0.4333 - accuracy: 0.8221
Epoch 9/20
 - 17s - loss: 0.3980 - accuracy: 0.8390
Epoch 10/20
 - 17s - loss: 0.3764 - accuracy: 0.8497
Epoch 11/20
 - 17s - loss: 0.3504 - accuracy: 0.8585
Epoch 12/20
 - 16s - loss: 0.3328 - accuracy: 0.8660
Epoch 13/20
 - 17s - loss: 0.3115 - accuracy: 0.8763
Epoch 14/20
 - 17s - loss: 0.3051 - accuracy: 0.8771
Epoch 15/20
 - 17s - loss: 0.2877 - accuracy: 0.8859
Epoch 16/20
 - 17s - loss: 0.2709 - accuracy: 0.8922
Epoch 17/20
 - 16s - loss: 0.2595 - accuracy: 0.8953
Epoch 18/20
 - 17s - loss: 0.2544 - accuracy: 0.8958
Epoch 19/20
 - 17s - loss: 0.2406 - accuracy: 0.9006
Ep

#### 1. Save the model and use the saved model to predict on new text data.

In [0]:
model.save('Vandith.h5')

In [28]:
from keras.models import load_model
model= load_model('Vandith.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [29]:
import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing']
#vectorizing the tweet by the pre-fitted tokenizer instance
sentence = tokenizer.texts_to_sequences(sentence)
#padding the tweet to have exactly the same shape as `embedding_2` input
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 1:
  print("Neutral")
elif sentiment == 0:
  print("Negative")
elif sentiment == 2:
  print("Positive")
else:
  print("Can not be determined")

2
Positive


#### 2. Apply GridSearchCV on the source code provided in the class

In [30]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, y=Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 42s - loss: 0.8311 - accuracy: 0.6489
Epoch 1/1
 - 42s - loss: 0.8233 - accuracy: 0.6488
Epoch 1/1
 - 43s - loss: 0.8335 - accuracy: 0.6394
Epoch 1/1
 - 42s - loss: 0.8317 - accuracy: 0.6417
Epoch 1/1
 - 42s - loss: 0.8282 - accuracy: 0.6457
Epoch 1/2
 - 42s - loss: 0.8316 - accuracy: 0.6437
Epoch 2/2
 - 41s - loss: 0.6928 - accuracy: 0.7061
Epoch 1/2
 - 43s - loss: 0.8227 - accuracy: 0.6523
Epoch 2/2
 - 43s - loss: 0.6882 - accuracy: 0.7097
Epoch 1/2
 - 42s - loss: 0.8283 - accuracy: 0.6447
Epoch 2/2
 - 42s - loss: 0.6779 - accuracy: 0.7162
Epoch 1/2
 - 42s - loss: 0.8285 - accuracy: 0.6398
Epoch 2/2
 - 41s - loss: 0.6822 - accuracy: 0.7088
Epoch 1/2
 - 43s - loss: 0.8239 - accuracy: 0.6449
Epoch 2/2
 - 42s - loss: 0.6749 - accuracy: 0.7176
Epoch 1/1
 - 21s - loss: 0.8380 - accuracy: 0.6365
Epoch 1/1
 - 21s - loss: 0.8285 - accuracy: 0.6433
Epoch 1/1
 - 21s - loss: 0.8344 - accuracy: 0.6438
Epoch 1/1
 - 21s - loss: 0.8362 - accuracy: 0.6360
Epoch 1/1
 - 21s - loss: 0.8341

#### 3. Apply the code on spamdata set availablein thesourcecode (text classification on the spam.csvdata set)

In [0]:
data1 = pd.read_csv('drive/My Drive/Python Colab/DL_Lesson5/spam.csv',encoding="ISO-8859-1")

# Keeping only the neccessary columns
data1 = data1[['v1','v2']]

In [0]:
data1['v2'] = data1['v2'].apply(lambda x: x.lower())
data1['v2'] = data1['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data1.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['v2'].values)
X = tokenizer.texts_to_sequences(data1['v2'].values)

X = pad_sequences(X)

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data1['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
def createmodel1():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='sigmoid'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

In [46]:
batch_size = 32
model1 = createmodel1()
model1.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 32s - loss: 0.1901 - accuracy: 0.9362
Epoch 2/5
 - 32s - loss: 0.0452 - accuracy: 0.9869
Epoch 3/5
 - 32s - loss: 0.0254 - accuracy: 0.9920
Epoch 4/5
 - 32s - loss: 0.0132 - accuracy: 0.9965
Epoch 5/5
 - 32s - loss: 0.0082 - accuracy: 0.9973
0.07656967166336198
0.9820554852485657
['loss', 'accuracy']
